In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
import pandas as pd
import pickle
import lfpykit
#from lfpykit.eegmegcalc import NYHeadModel
from lfpykit.eegmegcalc import FourSphereVolumeConductor
from lfpykit.eegmegcalc import InfiniteHomogeneousVolCondMEG as MEG
#import LFPy
from scipy import signal as ss

In [2]:
%matplotlib inline

In [3]:
def bandPassFilter(signal,low=0.1, high=130.):
	order = 2
	b, a = ss.butter(order, [low,high],btype='bandpass',fs=fs)
	y = ss.filtfilt(b, a, signal)
	return y

# Calculate EEG/MEG signals from simulation data

In [ ]:
file = open('data/Resting_state/samn_ASSR_wE_1_5_wI_1_0_data.pkl','rb')
data = pickle.load(file)

In [ ]:
dp = data['simData']['dipoleSum']

In [ ]:
file = open('data/Resting_state/spontaneous_data.pkl','rb')
data = pickle.load(file)

In [ ]:
dp = data['simData']['dipoleSum']
print(np.shape(dp))

## EEG Signal

In [ ]:
# FOUR SPHERE MODEL
radii = [79000., 80000., 85000., 90000.]  # (µm)
sigmas = [0.47, 1.71, 0.02, 0.41]  # (S/m) from Mazza et al., PLoS Comp Biol, 2023
r_electrodes = np.array([[0., 0., 90000.]]) # (µm)
pos = np.array([0., 78000.,0.]) # That's 725um in depth, check other depths!
sphere_model = FourSphereVolumeConductor(r_electrodes,radii, sigmas)
# # compute potential
signal = sphere_model.get_dipole_potential(dp.transpose(), pos)  # (mV)

In [ ]:
fs = 10000
s = int(4 * fs)
e = int(4.5 * fs)

eeg = signal[0]
times = np.arange(0,6000,6000/len(eeg)) # adjust to your simulation times

In [ ]:
#######################
# Plot dp moments & EEG
#######################

fig, axes = plt.subplots(figsize=(12,14),ncols=1,nrows=4)


axes[0].plot(times[s:e],eeg[s:e], lw=0.5) 
# plot dipolemoments 
for dp_comp in [0,1,2]: # x, y, z components of dipole
        axes[dp_comp+1].plot(times[s:e],dp[s:e,dp_comp], lw=.5)
    
# visual formatting
[ax.spines[spine].set_visible(False) for spine in ['top','right'] for ax in axes]
[ax.set_xlabel('Time (ms)',fontsize=16) for ax in axes]


axes[0].set_ylabel('EEG potential (mV)',fontsize=16)
axes[1].set_ylabel('X dp moment (nA*µm)',fontsize=16)
axes[2].set_ylabel('Y dp moment (nA*µm)',fontsize=16)
_ = axes[3].set_ylabel('Z dp moment (nA*µm)',fontsize=16)

#plt.savefig('EEG-Dipole-components-A1model.png',dpi=200)

In [ ]:
#######################
# Calc & plot PSD
#######################

fig,axes = plt.subplots(figsize=(10,5))

ts = signal.flatten()[4*fs:] # cut transient
nperseg = fs*3

freq_wel, ps_wel = ss.welch(ts,fs=fs,nperseg=nperseg)


axes.plot(freq_wel,ps_wel)
#axes.loglog(freq_wel,ps_wel)
axes.set_xlim(0,30)

axes.set_ylabel("Power",fontsize=16)
axes.set_xlabel('Frequency (Hz)',fontsize=16)
[axes.axvline(l,ls='--',color='k',alpha=.5) for l in [38,42]]
_ = [axes.spines[spine].set_visible(False) for spine in ['top','right']]

#plt.savefig('EEG-Spectrum-A1model.png',dpi=200)

## MEG Signal

In [ ]:
sensor_locations = np.array([[1E4,0,1E4],[-1E4,0,-1E4],[-1E4,0,1E4],[1E4,0,-1E4]])
meg = MEG(sensor_locations)
M = meg.get_transformation_matrix(pos)
H = M @ dp.transpose() 

In [ ]:
print(np.shape(H))

In [ ]:
sensor1 = H[0]
sensor2 = H[1]
sensor3 = H[2]
sensor4 = H[3]

In [ ]:
signal1 = np.empty(np.shape(sensor1)[1])
for i in range(np.shape(sensor1)[1]):
    signal1[i] =  np.linalg.norm(sensor1[:,i])

signal2 = np.empty(np.shape(sensor2)[1])
for i in range(np.shape(sensor2)[1]):
    signal2[i] =  np.linalg.norm(sensor2[:,i])

signal3 = np.empty(np.shape(sensor3)[1])
for i in range(np.shape(sensor3)[1]):
    signal3[i] =  np.linalg.norm(sensor3[:,i])

signal4 = np.empty(np.shape(sensor4)[1])
for i in range(np.shape(sensor4)[1]):
    signal4[i] =  np.linalg.norm(sensor4[:,i])

In [ ]:
meg_signal = np.sum([signal1,signal2,signal3,signal4],axis=0)/4. 

In [ ]:
fig, axes = plt.subplots(figsize=(12,20),ncols=1,nrows=5)


axes[0].plot(times[s:e],signal1[s:e], lw=0.5)
axes[1].plot(times[s:e],signal2[s:e], lw=0.5)
axes[2].plot(times[s:e],signal3[s:e], lw=0.5)
axes[3].plot(times[s:e],signal4[s:e], lw=0.5)
axes[4].plot(times[s:e],meg_signal[s:e], lw=0.5)

    
# visual formatting
[ax.spines[spine].set_visible(False) for spine in ['top','right'] for ax in axes]
[ax.set_xlabel('Time (ms)',fontsize=16) for ax in axes]


axes[0].set_ylabel('MEG amplitude sensor1 (?)',fontsize=14)
axes[1].set_ylabel('MEG amplitude sensor2 (?)',fontsize=14)
axes[2].set_ylabel('MEG amplitude sensor3 (?)',fontsize=14)
axes[3].set_ylabel('MEG amplitude sensor4 (?)',fontsize=14)
axes[4].set_ylabel('MEG amplitude average (?)',fontsize=14)

In [ ]:
#######################
# Calc & plot PSD
#######################

fig,axes = plt.subplots(figsize=(10,5))

ts = meg_signal.flatten()[4*fs:] # cut transient
nperseg = fs*3

freq_wel, ps_wel = ss.welch(ts,fs=fs,nperseg=nperseg)


axes.plot(freq_wel,ps_wel)
#axes.loglog(freq_wel,ps_wel)
axes.set_xlim(0,50)



axes.set_ylabel("Power",fontsize=16)
axes.set_xlabel('Frequency (Hz)',fontsize=16)
[axes.axvline(l,ls='--',color='k',alpha=.5) for l in [38,42]]
_ = [axes.spines[spine].set_visible(False) for spine in ['top','right']]